# Qu'est-ce qu'il y a dans ce notebook ?

Il y a toutes les fonctions permettant l'analyse des données obtenues après le POS_Taaging

1. Calcul des fréquences d'une liste de mots
2. Récupérer les différentes formes d'un lemme

In [ ]:
import json
from collections import Counter
import collections
import pandas as pd
from ipynb.fs.defs.fonctions_preprocess import serialisation_data
from ipynb.fs.defs.fonctions_preprocess import open_file
import re

## Ouverture des fichiers Json

In [ ]:
nouns = open_file("nouns.json")

In [ ]:
def extract_nouns(tuple_list):
    """extract different forms from tuple
    -> tuple list
    <- noun list"""
    
    form_list = []
    
    for tup in tuple_list:
        form_list.append(tup[0])
        
    return form_list

In [ ]:
nouns_list = extract_nouns(nouns)

## Calcul des fréquences des noms

### Calcul des occurences de chaque mot

Format Dictionnaire (objet Counter)

In [ ]:
def dict_occ (words_list) :
    """Count occurencies of words
    -> list of words
    <- dictionary of word frequencies like {word_1 : occurence, word_2 : occurence, ... , word_n : occurence}"""

    occ = collections.Counter(words_list)
    
    return occ

In [ ]:
occ = dict_occ(nouns_list)

print("Nombre d‘occurences uniques :", len(occ))

In [ ]:
ordered_occ_dict = sorted(occ.items(), key=lambda x:x[1], reverse = True)

print(ordered_occ_dict[0:50])
sort_dict = dict(ordered_occ_dict)
print(len(ordered_occ_dict))

### Calcul des fréquences et tri par fréquence les plus élevées

Stockée sous forme de dictionnaire

In [ ]:
def dict_freq (dict_occurence):
    """calculates frequencies of words
    <- dictionnary of occurences
    -> dictionary of frequencies like {word_1 : frequence, word_2 : frequence, ... , word_n : frequence}"""

    data_fq = {key : dict_occurence[key]/len(dict_occurence) for key in dict_occurence}
    
    return data_fq


In [ ]:
freq = dict_freq(occ)

#### Tri du dictionnaire par fréquence les plus élevées

In [ ]:
ordered_freq_dict = sorted(freq.items(), key=lambda x:x[1], reverse = True)

print(ordered_freq_dict[0:50])
sortdict = dict(ordered_freq_dict)

## Récupérer les différentes formes d'un mot

In [ ]:
def get_lemma_form (tuple_list):
    """creates a dictionnary of different forms for each lemma of the processed document
    -> liste of tuples like [(word_form1, word_lemma1), ..., (word_formn, word_lemman)
    <- dictionnary with as key a lemma and as values the different forms found in the text
    dictionary form : {lemma1 : [form1,form2, formn], lemma2 : [form1,form2,formn], ..., lemman: [forma1,form2,formn]}
    """
    
    lemma_and_form = {}
    
    for tup in tuple_list:
        if tup[1] not in lemma_and_form: lemma_and_form[tup[1]] = []
        if tup[0] not in lemma_and_form[tup[1]]: lemma_and_form[tup[1]].append(tup[0])
        
    return lemma_and_form

In [ ]:
lemma_form = get_lemma_form(nouns)

In [ ]:
print("Nombre de lemme différents : ", len(lemma_form))

### Sérialisation des données

Via la fonction serialisation_data de POS_tagging.ipynb

In [ ]:
def serialisation_data (data, title):
  """
  Serialize data in a json file
  -> Title mus be a string : title.json
  <- Save a file in desktop
  """

  with open(title, "w+") as file:
    json.dump(data, file)

In [ ]:
#Sérialisation du dictionnaire d'occurence
#serialisation_data (occ, "dict_occ.json")

#Sérialisation du dictionnaire de fréquence
#serialisation_data (freq, "dict_freq.json")

#Sérialisation du dictionnaire de fréquence trié
serialisation_data (sortdict, "dict_ord_freq.json")

#Sérialisation du dictionnaire de lemmes
#serialisation_data (lemma_form, "lemme_form.json")

## Création de fichiers CSV à partir d'un dictionnaire

Permet de faciliter la lecture des données

In [ ]:
data_frame = pd.DataFrame.from_dict(lemma_form, orient='index')

print(data_frame)
col = data_frame.shape[1]

print(col)

data_frame.to_csv(r"./ lemmes.csv", header = [i for i in range(1,col+1)])

In [ ]:
data_frame = pd.DataFrame.from_dict(freq, orient='index')

print(data_frame)
col = data_frame.shape[1]

print(col)

data_frame.to_csv(r"./ fréquences.csv", header = ["Fréquence"])

In [ ]:
data_frame = pd.DataFrame.from_dict(sortdict, orient='index')

print(data_frame)

data_frame.to_csv(r"./ fréquences_ordered.csv", header = ["Fréquence"])